In [204]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np
import functools
from heapq import heappop, heappush

In [205]:
# def action struct
action = namedtuple('Action', ['pos1', 'pos2'])

PUZZLE_DIM = 3
RANDOMIZE_STEPS = 100_000

In [206]:
# list of available actions
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)] 
    actions = []
    if x > 0: # can go up
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:  # can go down
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:  # can go left
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:  # can go right
        actions.append(action((x, y), (x, y + 1)))
    return actions


def counter(fn):
    @functools.wraps(fn)
    def helper(*args, **kargs):
        helper.calls += 1
        return fn(*args, **kargs)

    helper.calls = 0
    return helper

@counter
def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state


In [207]:
# function to calculate the manhattan distance (distance between the goal and the current state)
def manhattan_distance(state: np.ndarray) -> int:
    distance = 0
    for value in range(1, PUZZLE_DIM ** 2):
        x, y = np.where(state == value)
        x, y = int(x[0]), int(y[0])
        goal_x, goal_y = divmod(value, PUZZLE_DIM)
        distance += abs(x - goal_x) + abs(y - goal_y)
    return distance

# bool function that return true if the state is the goal
def goal_function( state ,goal ):
    label = np.array_equal(state, goal)
    if(label):
        return True
    else:
        return False

def a_star_search(initial_state: np.ndarray, goal: np.ndarray) -> list['Action']:
    frontier = [(manhattan_distance(initial_state), 0, tuple(initial_state.flatten()), [])]
    visited = set()
    
    while frontier:
        _, real_cost, state_tuple, path = heappop(frontier)
        
        state = np.array(state_tuple).reshape(PUZZLE_DIM, PUZZLE_DIM)
        
        if goal_function(state, goal):
            return path
        
        if state_tuple in visited:
            continue
        
        
        visited.add(state_tuple)
        
        for act in available_actions(state):
            new_state = do_action(state, act)
            new_path = path + [act]
            new_real_cost = real_cost + 1
            new_est_cost = new_real_cost + manhattan_distance(new_state)
            heappush(frontier, (new_est_cost, new_real_cost, tuple(new_state.flatten()), new_path))
    
    # No solution found
    return None


In [208]:
RANDOMIZE_STEPS = 100_000
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
goal = state.copy()
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))

Randomizing: 100%|██████████| 100000/100000 [00:00<00:00, 344239.82it/s]


In [209]:
solution = a_star_search( state, goal)
print(len(solution))
print(solution)
print(do_action.calls) 

18
[Action(pos1=(0, 0), pos2=(0, 1)), Action(pos1=(0, 1), pos2=(0, 2)), Action(pos1=(0, 2), pos2=(1, 2)), Action(pos1=(1, 2), pos2=(1, 1)), Action(pos1=(1, 1), pos2=(0, 1)), Action(pos1=(0, 1), pos2=(0, 0)), Action(pos1=(0, 0), pos2=(1, 0)), Action(pos1=(1, 0), pos2=(2, 0)), Action(pos1=(2, 0), pos2=(2, 1)), Action(pos1=(2, 1), pos2=(2, 2)), Action(pos1=(2, 2), pos2=(1, 2)), Action(pos1=(1, 2), pos2=(0, 2)), Action(pos1=(0, 2), pos2=(0, 1)), Action(pos1=(0, 1), pos2=(0, 0)), Action(pos1=(0, 0), pos2=(1, 0)), Action(pos1=(1, 0), pos2=(1, 1)), Action(pos1=(1, 1), pos2=(2, 1)), Action(pos1=(2, 1), pos2=(2, 2))]
142371
